## **Import the TensorFlow library**

This code contains a small script that is used for generating a folder full of images. The idea is to show these images to the OpenMV camera when the training with the OL methos is required. Since the idea is to apply a supervised training is required to have a ground truth forthe computation of the error that the model is doing. This script fills a directory with images of digits and creates a txt file in which all the labels are saved. 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image

import numpy as np
import pandas as pd
import seaborn as sns
import random 
import csv 
import sys
import os
import re
from random import seed

import time
#import os, os.path
import cv2
import glob
from keras import applications
#from keras.applications import vgg19
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score

from numpy.ma.core import size

import torchvision.models as models
from torchvision import transforms
ROOT_PATH = os.path.abspath('')
sys.path.insert(0, ROOT_PATH + '/lib')

# VS Code

# IMPORT AND LOAD MODELS

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans


import random 
from random import seed

import time


import numpy as np
import matplotlib.pyplot as plt


# Load features and labels
features_saved = np.loadtxt('Models/Original_model/ll_features_10.txt')
labels_features_saved = np.loadtxt('Models/Original_model/ll_labels_features_10.txt').astype(int)

# Load model - CFR
model = keras.models.load_model('Models/Original_model/mnist_cnn.h5') # Frozen model 

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')



# Test import
# from lib import simulation_lib
# from lib.simulation_lib import

from lib import Kmeans_lib
from lib.Kmeans_lib import *
from lib.EvalMetrics import *


2022-10-20 16:52:33.828725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-20 16:52:41.592288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# LOAD DATASETS

In [2]:
(data_train, label_train),(data_test, label_test) = mnist.load_data() # Load data
print('The original dataset shapes from MNIST are')
print(f'    Train dataset shape: {data_train.shape}')
print(f'    Test dataset shape:  {data_test.shape}')

The original dataset shapes from MNIST are
    Train dataset shape: (60000, 28, 28)
    Test dataset shape:  (10000, 28, 28)


In [3]:
train_samples = label_train.shape[0]
test_samples  = label_test.shape[0]
img_rows, img_cols = 28, 28

trainLow_samples = np.sum(np.where(label_train < 6, 1, 0))
testLow_samples  = np.sum(np.where(label_test  < 6, 1, 0))

# separate in containers data that is lower and higer than 6
# TRAIN - LOW
data_low_train   = np.zeros([trainLow_samples,28,28])
label_low_train  = np.zeros(trainLow_samples)
#       - HIGH
data_high_train  = np.zeros([train_samples-trainLow_samples,28,28])
label_high_train = np.zeros(train_samples-trainLow_samples)

# TEST - LOW
data_low_test   = np.zeros([testLow_samples,28,28])
label_low_test  = np.zeros(testLow_samples)

#      - HIGH
data_high_test  = np.zeros([test_samples-testLow_samples,28,28])
label_high_test = np.zeros(test_samples-testLow_samples)

j,k = 0,0
for i in range(0,train_samples):  
    if(label_train[i]<6):
        data_low_train[j,:,:] = data_train[i,:,:]
        label_low_train[j]    = label_train[i]
        j+=1
    else:
        data_high_train[k,:,:] = data_train[i,:,:]
        label_high_train[k]    = label_train[i]
        k+=1


j,k = 0,0
for i in range(0,test_samples):
    if(label_test[i]>5):
        data_high_test[k,:,:] = data_test[i,:,:]
        label_high_test[k]    = label_test[i]
        k+=1  
    else:
        data_low_test[j,:,:] = data_test[i,:,:]
        label_low_test[j]    = label_test[i]
        j+=1

# Reshape arrays
data_low_train  = data_low_train.reshape(data_low_train.shape[0], img_rows, img_cols, 1)
data_high_train = data_high_train.reshape(data_high_train.shape[0], img_rows, img_cols, 1)
data_low_test   = data_low_test.reshape(data_low_test.shape[0], img_rows, img_cols, 1)
data_high_test  = data_high_test.reshape(data_high_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
print('After the separation of the dataset in groups higer and lower/equal than 6 the datasets are:')
print(f'     Train dataset lower than 6 has shape:  {data_low_train.shape}')
print(f'     Train dataset higher than 6 has shape: {data_high_train.shape}')
print()
print(f'     Test dataset lower than 6 has shape:  {data_low_test.shape}')
print(f'     Test dataset higher than 6 has shape: {data_high_test.shape}')

# Normalize the colors from 0-255 to 0-1
data_low_train  = data_low_train.astype(np.float32) / 255.0
data_high_train = data_high_train.astype(np.float32) / 255.0
data_low_test   = data_low_test.astype(np.float32) / 255.0
data_high_test  = data_high_test.astype(np.float32) / 255.0

After the separation of the dataset in groups higer and lower/equal than 6 the datasets are:
     Train dataset lower than 6 has shape:  (36017, 28, 28, 1)
     Train dataset higher than 6 has shape: (23983, 28, 28, 1)

     Test dataset lower than 6 has shape:  (6031, 28, 28, 1)
     Test dataset higher than 6 has shape: (3969, 28, 28, 1)


In [4]:
n_samples = 100 # 400
digits_train = np.zeros((n_samples,28,28))
digits_test = np.zeros((n_samples,28,28))
label_digits_train = np.zeros(n_samples)
label_digits_test = np.zeros(n_samples)

for i in range(0, n_samples):
  n = random.randint(0,len(data_train)-1)
  digits_train[i,:,:] = data_train[n,:,:]
  label_digits_train[i] = label_train[n]
  m = random.randint(0,len(data_test)-1)
  digits_test[i,:,:] = np.copy(data_test[m,:,:])
  label_digits_test[i] = label_test[m]


digits_train  = digits_train.reshape(digits_train.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
digits_test = digits_test.reshape(digits_test.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
input_shape = (img_rows, img_cols, 1)

print(f'The dataset on which the model will be TRAINED has shape {digits_train.shape}') # da cambiare con il nuovo set
print(f'The dataset on which the model will be TESTED has shape  {digits_test.shape}') # da cambiare con il nuovo set

The dataset on which the model will be TRAINED has shape (100, 28, 28, 1)
The dataset on which the model will be TESTED has shape  (100, 28, 28, 1)


# FUNCTIONS

# TRAIN THE MODEL ON THE DIGITS 6-9 (OL METHOD)

In [12]:
def run_frozen():
    pass


''' Function to check if the current label is already known to the model (OL layer). If not it augments the custom layer adding a new node'''
def CheckLabelKnown(model, current_label):
    
    found = False
    for i in range(0, len(model.label)):
        if(current_label == model.label[i]):
            found = True
        
    if not found:
        print(f'New digit detected ->', current_label)

        model.label.append(current_label)   # Add new digit to label
                
        # Increase weights and biases dimensions
        model.W = np.hstack((model.W, np.zeros([model.W.shape[0],1])))
        model.b = np.hstack((model.b, np.zeros([1])))
        
        model.W_2 = np.hstack((model.W_2, np.zeros([model.W.shape[0],1])))
        model.b_2 = np.hstack((model.b_2, np.zeros([1])))


        


def ComputeEvalMetrics(true_label, pred_label, labels_list):

    print(true_label.shape)
    print(pred_label.shape)
    confusion = confusion_matrix(true_label, pred_label, labels = labels_list)
    print('Confusion Matrix\n')
    print(confusion)

    print('\nAccuracy: {:.2f}\n'.format(accuracy_score(true_label, pred_label)))

    print('Micro Precision: {:.2f}'.format(precision_score(true_label, pred_label, average='micro')))
    print('Micro Recall: {:.2f}'.format(recall_score(true_label, pred_label, average='micro')))
    print('Micro F1-score: {:.2f}\n'.format(f1_score(true_label, pred_label, average='micro')))

    print('Macro Precision: {:.2f}'.format(precision_score(true_label, pred_label, average='macro')))
    print('Macro Recall: {:.2f}'.format(recall_score(true_label, pred_label, average='macro')))
    print('Macro F1-score: {:.2f}\n'.format(f1_score(true_label, pred_label, average='macro')))

    print('Weighted Precision: {:.2f}'.format(precision_score(true_label, pred_label, average='weighted')))
    print('Weighted Recall: {:.2f}'.format(recall_score(true_label, pred_label, average='weighted')))
    print('Weighted F1-score: {:.2f}'.format(f1_score(true_label, pred_label, average='weighted')))

    print('\nClassification Report\n')
    print('TO DO')
    # print(classification_report(true_label, pred_label, target_names=['A', 'E', 'I', 'O', 'U']))

In [6]:
'''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):

  # Define initial set of features
  labels_init_list = list(range(0, n_cluster))

  # labels_init_list = list([1, 9, 5, 0])
  # labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  # n_cluster = len(labels_init_list)

  # Extract from the saved features the labels that we need
  features_saved_init = []
  labels_features_saved_init = []
  # Extract features of digits considered in labels_init_list
  for i in range(0, len(features_saved)):
      if labels_features_saved[i] in labels_init_list:
        features_saved_init.append(features_saved[i,:])
        labels_features_saved_init.append(labels_features_saved[i])
  
  # Convert list to nparray
  features = np.array(features_saved_init)
  features = features.astype('float32')
  labels_features = np.array(labels_features_saved_init)  

  # Concateno al vettore delle features iniziali le features della nuova batch da analizzare
  features = np.concatenate((features, features_run))
  labels_features = np.append(labels_features, labels_features_run).astype(int)

  # Repeat until clustering is correct
  while True:
    # KMean Clustering
    k_mean = create_k_mean(features, n_cluster)

    # Find pseudolabels for each new image
    # Pseudolabels are computed by looking at the confusion matrix of the saved dataset (where ground truth is known)
    clusters_features_saved = list(k_mean.labels_[0:len(labels_features_saved_init)])
    labels_features_saved_init = list(labels_features_saved_init)
    cluster_list = list(range(0,n_cluster))
    map_clu2lbl, map_lbl2clu = cluster_to_label(clusters_features_saved, labels_features_saved_init, cluster_list, labels_init_list) 
 
    if len(map_clu2lbl) == n_cluster:
        # Exit the loop
        break

  clusters_features = k_mean.labels_

  # Find pseudo label (labels obtained from the model of each image
  # pseudolabels = np.zeros(len(clusters_features), dtype=int)

  #print("cluster features", clusters_features)
  #print("labels features", labels_features)


  pseudolabels = []

  # Compute pseudolabels only for new digits
  err = 0 # Initialize error counter
  for i in range(len(clusters_features) - batch_size, len(clusters_features)):
    pseudolabel = map_clu2lbl[clusters_features[i]]
    pseudolabels.append(pseudolabel)


    # print(pseudolabels_features[i], labels_features[i], clusters_features[i])
    if pseudolabel != labels_features[i]:
      err += 1
  
  # print(pseudolabels_features)
  # print(labels_features[len(clusters_features)-batch_size:len(clusters_features)])

  # conf_matrix(clusters_features_saved, labels_features_saved_init, , labels_init_list)
  
  # Evaluation metrics
  # ComputeEvalMetrics(labels_features, pseudolabels_features, cluster_list, labels_init_list)

  return pseudolabels, err


########################################################################################################
########################################################################################################
def update_active_layer(model, features, pseudolabel):

    learn_rate = model.l_rate

    CheckLabelKnown(model, pseudolabel)
    
    y_true_soft = DigitToSoftmax(pseudolabel, model.label)
               
    # Prediction
    y_pred = model.predict(features)
        
    # Backpropagation
    cost = y_pred-y_true_soft
        
    for j in range(0,model.W.shape[0]):

         # Update weights
        dW = np.multiply(cost, features[j]*learn_rate)
        model.W[j,:] = model.W[j,:]-dW

    # Update biases
    db      = np.multiply(cost, learn_rate)
    model.b = model.b-db



########################################################################################################
########################################################################################################
def trainOneEpoch_OL(model, images, labels, features_saved, labels_features_saved, batch_size):
       
    learn_rate = model.l_rate
    n_cluster = 10
    n_samples = images.shape[0]

    # BATCH PROCESSING OF DATA
    n_batch = int(np.ceil(n_samples / batch_size))
    images_batch = np.array_split(images, n_batch)
    labels_batch = np.array_split(labels, n_batch)

    err_tot = 0
    pseudo_labels = []
    for i in range(0, n_batch):
        print("Starting batch: {}/{}".format(i+1, n_batch))
        # Features extraction
        start1 = time.time()
        features_batch = model.ML_frozen.predict(images_batch[i].reshape((batch_size,28,28,1)), verbose = False)
        end1 = time.time()

        # Kmean clustering
        start2 = time.time()
        pseudo_labels_batch, err = k_mean_clustering(features_batch, features_saved, labels_batch[i], labels_features_saved, n_cluster, batch_size)
        end2 = time.time()
        pseudo_labels.extend(pseudo_labels_batch)
        err_tot += err

        print("Features extraction took {:.3f} seconds and Kmean clustering took {:.3f} seconds, with {:.1%} accuracy ({} errors)".format(end1-start1, end2-start2, 1-err/batch_size, err))

    print("Total clustering error: {:.1%} ({}/{} errors)".format(err_tot/n_samples, err_tot, n_samples))

    # ONLINE-LEARNING -> si può spostare nella parte delle batch
    print('**********************************\n Performing training with OL\n')
    features_images = model.ML_frozen.predict(images.reshape((n_samples,28,28,1)), verbose = False)
    for i in range(0, n_samples):
        update_active_layer(model, features_images[i,:], pseudo_labels[i])


In [7]:
Model_OL = Custom_Layer(model)
Model_OL.title      = 'OL'
Model_OL.filename   = 'OL'
Model_OL.l_rate     = 0.01
Model_OL.batch_size = 8

batch_size = 100 # 20

#import importlib
#importlib.reload(Kmeans_lib)

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"  

digits = np.concatenate((digits_train, digits_test))   
labels = np.concatenate((label_digits_train, label_digits_test))  
    
trainOneEpoch_OL(Model_OL, digits, labels, features_saved, labels_features_saved, batch_size)

Starting batch: 1/2
Features extraction took 0.214 seconds and Kmean clustering took 0.742 seconds, with 78.0% accuracy (22 errors)
Starting batch: 2/2
Features extraction took 0.070 seconds and Kmean clustering took 11.161 seconds, with 67.0% accuracy (33 errors)
Total clustering error: 27.5% (55/200 errors)
**********************************
 Performing training with OL

new digit detected 8
new digit detected 6
new digit detected 7
new digit detected 9


In [8]:
feat = features_saved[1,:]

##(features_saved.all)
#max(features_saved)

feat.dtype

np.amin(feat)
np.amax(feat)

feat.shape
feat

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.042, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.245, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.007, 0.715, 0.   , 0.   , 0.   , 0.524, 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.777, 0.053, 0.9  , 0.   , 0.   ,
       0.   , 0.972, 0.   , 0.   , 0.   , 1.784, 0.   , 0.   , 1.024,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.714, 2.935, 0.   ,
       0.   , 0.988, 2.111, 0.   , 0.   , 1.147, 0.   , 0.   , 0.463,
       1.97 , 0.   , 2.253, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.163, 0.698, 2.776, 0.   , 0.   , 0.   , 0.   , 2.963, 0.   ,
       1.29 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 2.057,
       0.   , 0.   , 0.   , 0.462, 0.017, 0.   , 0.   , 0.   , 0.   ,
       0.278, 0.   , 0.   , 0.   , 0.   , 0.   , 0.686, 1.092, 0.   ,
       2.389, 0.   , 0.   , 1.416, 0.   , 1.505, 0.   , 1.37 , 0.   ,
       0.   , 0.   ,

In [9]:
X = digits_train[1,:]
YY = Model_OL.ML_frozen.predict(X.reshape((1,28,28,1)), verbose = False)

# YY2 = np.array(YY as features_saved)

np.amin(YY)
np.amax(YY)

YY.shape

np.set_printoptions(suppress=True)

YY

array([[0.9676225 , 0.12028278, 0.        , 0.        , 1.3939843 ,
        0.20071982, 0.        , 0.7091535 , 1.1316229 , 0.        ,
        3.507143  , 0.        , 0.        , 0.        , 1.6523658 ,
        2.260346  , 0.        , 0.4986054 , 0.36237246, 3.3724933 ,
        0.        , 0.        , 0.38916868, 1.3088677 , 2.1849537 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.8348079 , 0.        , 2.8453293 ,
        0.        , 0.        , 0.        , 0.        , 1.8739043 ,
        0.        , 0.        , 4.577672  , 0.        , 1.3842472 ,
        0.08130443, 4.681705  , 0.6974365 , 0.        , 3.9124522 ,
        1.738846  , 1.5231588 , 0.        , 0.84488916, 1.6516736 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 6.088375  ,
        0.        , 0.        , 0.        , 0.50

In [10]:



#from simulation_lib import *

#plot_barChart(Model_OL)
#plot_confMatrix(Model_OL)
#plot_table(Model_OL)

# from lib import simulation_lib as sim_lib
# sim_lib.plot_barChart(Model_OL)
# sim_lib.plot_confMatrix(Model_OL)
# sim_lib.plot_table(Model_OL)

In [11]:



############

# Dummy  main for debug
if False:
  n = range(0, 10)
  images = data_train[n,:,:]
  labels_features_run = label_train[n]

  features_run = []
  for img in images:
    features_run.append(Custom_Layer(model).ML_frozen.predict(img.reshape((1,28,28,1)))/1000)

  features_run = np.array(features_run).squeeze(axis=1)

  n_cluster = 5 # ARGOMENTO NON USATO
  # AGGIORNARE PASSANDO LA LISTA DI LABELS DA CLUSTERIZZARE
  batch_size = 10

  kmean = k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size)